# COGS 108 - Final Project 

# Overview

The research conducted focuses on the effect of time on the upvotes generated for a given post on Reddit. We investigate the optimal time of day, and day of the week to post on 'r/funny' that would generate the greatest amount of upvotes, comments and views that readily predict a high karma count. In order to do so, we ran a script for 24 hours that collected the first 100 posts made in the 'new' section of r/funny at midnight and followed those 100 posts for 24 hours and tracked their upvotes and comments every 30 minutes. We also kept track of online users every 30 minutes to see if that affected upvotes and comments. The collected data was stored in csv files.

The findings of this research question are useful beyond the trivial matter of boosting upvotes, they are valuable for businesses that use Reddit for marketing, for election posts that desire the greatest audience, for the greatest exposure to breaking news and controversial matters, etc. The implications of the research question at hand are great and can be further applied when additional subreddits are integrated. 

Our findings suggest that it is difficult to pinpoint a specific best day but we have results for best time to post per day for the r/funny subreddit.

# Names

- Cyrus Shen
- Randy Vo
- Kimberly Chuong
- Josue Ayala
- Conner Henderson
- Mykel Rodriguez

# Group Members IDs

- A13370306
- A12759508
- A15022184
- #########
- A12855214
- A########

# Research Question

What day of the week and time of day is optimal for maximizing the amount of karma for a given post on the r/funny subreddit of Reddit?

## Background and Prior Work

To run analysis on reddit data, we first had to gain permission to use Reddit's API. The API is readily accessibe after registering for use, and is easily manipulated using the Reddit API wrapper, Python praw. 
References (include links):
- 1)
- 2)

# Hypothesis


We predict that the weekends will be more effective for upvotes than weekdays and that evenings will be more effective than mornings and afternoons. Specifically, we predict that Saturday at 9am will have the most posts overall with the highest upvotes and comments. 

# Dataset(s)

## Reddit API

The Reddit API is a service offered by the official Reddit website that allows access to extracting a multitude of its available information. For this project, we scraped data for the the first 100 posts from the 'New' section of the 'r/funny' subreddit and followed those posts for 24 hours (data broken down into 30 minute intervals) across all 7 days of the week, while collecting information on the number of upvotes, number of comments, and number of users online at corresponding time periods into csv files.

- Dataset Name: Reddit API
- Link to the dataset: https://www.reddit.com/wiki/api, https://github.com/reddit-archive/reddit/wiki/API
- Number of observations: Reddit API gives access to all submissions on their website, so there is an unlimited number of data   points we could gather. The datasets for one particular day of the week consists of 4900 observations for both number of       upvotes and number of comments, and 49 observations for number of users online.

# Setup

In [1]:
# Import libraries for data wrangling, data cleaning, and data visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Configure the imported libraries

# Display plots directly on the notebook
%matplotlib inline

# Display all columns and rows in DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Control figure aesthetics for seaborn
sns.set()
sns.set_context('talk')

## Reddit Scraping Code for Data Collection

In [3]:
#collect data for 24hrs
import time
from datetime import datetime
import praw
import pandas as pd

run = False #change this to True to run
reddit = praw.Reddit(client_id='eOqj6Q2BiQQyKQ', client_secret='1hqeAwOvlkY1PYw51ewhbrItd5k', user_agent='Reddit WebScrapping')

sub_title = 'funny' #title of subreddit to scrape

time2sleep = 1800 #how long the program should sleep before continuing
hours = 48 #how long(30 minute intervals) you want to run the script for
count = 0 #each count represents 1 interval passing

dt_object = datetime.fromtimestamp(time.time())
weekday = dt_object.strftime('%A')

fname_votes = './{}_votes.csv'.format(weekday)
fname_comments = './{}_comments.csv'.format(weekday)
fname_online = './{}_online_users.csv'.format(weekday)

votes = {} #store post ids and votes. Positive votes and NEGATIVE votes (when post.downs is > 0)
totalcomments = {} #store post ids and the corresponding number of comments
online = {'timestamp':[],'users_online':[]} #store number of online users and timestamp of subreddit

subred = reddit.subreddit(sub_title)

#get 100 posts from new
sub_posts = subred.new(limit=100)

#initialize the storage of the 100 posts that we are tracking
for post in sub_posts:
    pid = str(post.id)
    if pid not in votes: #if new post, add it to the dictionary
        votes[pid] = [] #initialize empty list
        totalcomments[pid] = [] #initialize empty list
    else:#just in case we get some repeating posts
        pass

while (count <= hours) and run:
    
    print('Starting 30min interval {}'.format(count))
    subred = reddit.subreddit(sub_title)
    #every 30minutes store number of online users in the subreddit
    online['timestamp'].append(time.time())
    online['users_online'].append(subred.active_user_count)
    
    for id_val in votes.keys():
        result = reddit.submission(id=id_val) #go to the specific post
        votes[id_val].append(result.ups - result.downs) #get the current number of upvotes and add to the list
        totalcomments[id_val].append(result.num_comments) #get number of comments
        
    #store into csv every 30min interval in case something goes wrong halfway through
    upvotes = pd.DataFrame(list(votes.values()),index=votes.keys())
    upvotes.to_csv(fname_votes,index_label='id')

    comments = pd.DataFrame(list(totalcomments.values()),index=totalcomments.keys())
    comments.to_csv(fname_comments,index_label='id')

    on = pd.DataFrame(list(online.values()),index=online.keys())
    on.to_csv(fname_online,index_label='timestamp')
    
    if count == hours:
        print('finished collecting data')
        break
    else:
        print('sleeping for {} seconds...'.format(time2sleep))
        time.sleep(time2sleep)
        count+=1



# Data Cleaning

Let's take a look at the csv file datasets we have collected. To start off, we check the upvotes, comments, and number of online users datasets for the day of Sunday. Datasets are the same layout across all days in their respective categories of upvotes, comments, and users online.

In [4]:
# Load Sunday_upvotes.csv file into a DataFrame
df_Sunday_upvotes = pd.read_csv('Sunday_upvotes.csv')

# Check the first few rows
df_Sunday_upvotes.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,dxjvx7,1,5,6,7,7,5,4,4,6,7,5,3,3,3,4,2,5,3,4,4,5,6,2,5,6,6,6,3,4,2,4,2,3,4,4,5,4,4,4,4,3,4,3,3,3,3,6,5,2
1,dxjvrw,2,18,26,27,26,33,36,36,37,36,37,34,35,38,35,37,34,34,35,39,39,34,37,35,34,39,35,33,38,37,37,38,33,38,39,38,35,35,36,33,36,37,37,33,38,36,37,34,36
2,dxjvlv,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,dxjvi3,4,27,34,55,66,69,77,85,92,102,111,124,125,128,132,126,129,128,133,138,135,140,139,144,144,144,142,146,146,148,146,149,147,147,149,153,151,147,146,152,148,149,149,145,149,148,152,149,149
4,dxjv8u,1,7,7,6,5,5,5,6,5,4,6,6,6,6,7,5,8,5,5,6,6,7,5,6,6,6,4,4,5,7,6,5,8,6,4,5,5,6,7,4,7,4,7,6,7,5,6,8,7


In [5]:
# Load Sunday_comments.csv file into a DataFrame
df_Sunday_comments = pd.read_csv('Sunday_comments.csv')
df_Sunday_comments.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,dxjvx7,0,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,dxjvrw,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,dxjvlv,0,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3,dxjvi3,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,dxjv8u,2,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [6]:
# Load Sunday_online_users.csv file into a DataFrame
df_Sunday_online_users = pd.read_csv('Sunday_online_users.csv', index_col = 0)
df_Sunday_online_users.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
timestamp,1.573978e+09,1.573979e+09,1.573981e+09,1.573983e+09,1.573985e+09,1.573987e+09,1.573988e+09,1.573990e+09,1.573992e+09,1.573994e+09,1.573996e+09,1.573998e+09,1.573999e+09,1.574001e+09,1.574003e+09,1.574005e+09,1.574007e+09,1.574009e+09,1.574010e+09,1.574012e+09,1.574014e+09,1.574016e+09,1.574018e+09,1.574019e+09,1.574021e+09,1.574023e+09,1.574025e+09,1.574027e+09,1.574029e+09,1.574030e+09,1.574032e+09,1.574034e+09,1.574036e+09,1.574038e+09,1.574039e+09,1.574041e+09,1.574043e+09,1.574045e+09,1.574047e+09,1.574049e+09,1.574050e+09,1.574052e+09,1.574054e+09,1.574056e+09,1.574058e+09,1.574060e+09,1.574061e+09,1.574063e+09,1.574065e+09
users_online,2.012800e+04,2.198900e+04,2.210100e+04,2.531800e+04,2.667100e+04,1.683600e+04,1.760700e+04,1.843800e+04,1.992500e+04,2.666500e+04,2.677800e+04,3.675000e+04,4.600300e+04,7.104600e+04,5.769500e+04,6.357800e+04,2.788000e+04,3.209100e+04,6.131300e+04,5.629900e+04,5.533500e+04,5.356600e+04,6.809000e+04,5.293300e+04,4.641200e+04,4.414000e+04,4.266400e+04,4.330800e+04,4.390300e+04,4.387400e+04,4.877100e+04,4.997800e+04,4.763000e+04,4.541000e+04,5.162000e+04,5.217000e+04,4.810800e+04,4.170900e+04,3.703400e+04,3.584300e+04,3.317400e+04,3.570100e+04,4.902900e+04,5.825500e+04,5.205100e+04,4.384900e+04,3.820600e+04,3.752000e+04,3.499100e+04


Looking at the datasets above, we see that there are no column names indicated, making the datasets overall not human reader-friendly. Therefore, some data cleaning needs to be done to increase the readability of these datasets.

In [7]:
# Create a variable to rename columns to time in 30 minute increments
time_cols = {'0': '12:00 am', '1': '12:30 am', '2': '1:00 am', '3': '1:30 am', '4': '2:00 am', '5': '2:30 am', '6': '3:00 am', '7': '3:30 am', '8':'4:00 am', '9': '4:30 am', '10': '5:00 am', '11': '5:30 am', '12': '6:00 am', '13': '6:30 am', '14': '7:00 am', '15': '7:30 am', '16': '8:00 am', '17': '8:30 am', '18': '9:00 am', '19': '9:30 am', '20': '10:00 am', '21': '10:30 am', '22': '11:00 am', '23': '11:30 am', '24': '12:00 pm', '25': '12:30 pm', '26': '1:00 pm', '27': '1:30 pm', '28': '2:00 pm', '29': '2:30 pm', '30': '3:00 pm', '31': '3:30 pm', '32': '4:00 pm', '33': '4:30 pm', '34': '5:00 pm', '35': '5:30 pm', '36': '6:00 pm', '37': '6:30 pm', '38': '7:00 pm', '39': '7:30 pm', '40': '8:00 pm', '41': '8:30 pm', '42': '9:00 pm', '43': '9:30 pm', '44': '10:00 pm', '45': '10:30 pm', '46': '11:00 pm', '47': '11:30 pm', '48': '11:59 pm'}

# Add/rename column names for Sunday upvotes
df_Sunday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Sunday_upvotes.rename(columns = time_cols, inplace = True)

df_Sunday_upvotes.head()

,id,12:00 am,12:30 am,1:00 am,1:30 am,2:00 am,2:30 am,3:00 am,3:30 am,4:00 am,4:30 am,5:00 am,5:30 am,6:00 am,6:30 am,7:00 am,7:30 am,8:00 am,8:30 am,9:00 am,9:30 am,10:00 am,10:30 am,11:00 am,11:30 am,12:00 pm,12:30 pm,1:00 pm,1:30 pm,2:00 pm,2:30 pm,3:00 pm,3:30 pm,4:00 pm,4:30 pm,5:00 pm,5:30 pm,6:00 pm,6:30 pm,7:00 pm,7:30 pm,8:00 pm,8:30 pm,9:00 pm,9:30 pm,10:00 pm,10:30 pm,11:00 pm,11:30 pm,11:59 pm
0,dxjvx7,1,5,6,7,7,5,4,4,6,7,5,3,3,3,4,2,5,3,4,4,5,6,2,5,6,6,6,3,4,2,4,2,3,4,4,5,4,4,4,4,3,4,3,3,3,3,6,5,2
1,dxjvrw,2,18,26,27,26,33,36,36,37,36,37,34,35,38,35,37,34,34,35,39,39,34,37,35,34,39,35,33,38,37,37,38,33,38,39,38,35,35,36,33,36,37,37,33,38,36,37,34,36
2,dxjvlv,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,dxjvi3,4,27,34,55,66,69,77,85,92,102,111,124,125,128,132,126,129,128,133,138,135,140,139,144,144,144,142,146,146,148,146,149,147,147,149,153,151,147,146,152,148,149,149,145,149,148,152,149,149
4,dxjv8u,1,7,7,6,5,5,5,6,5,4,6,6,6,6,7,5,8,5,5,6,6,7,5,6,6,6,4,4,5,7,6,5,8,6,4,5,5,6,7,4,7,4,7,6,7,5,6,8,7


In [8]:
# Add/rename column names for Sunday comments
df_Sunday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Sunday_comments.rename(columns = time_cols, inplace = True)

df_Sunday_comments.head()

,id,12:00 am,12:30 am,1:00 am,1:30 am,2:00 am,2:30 am,3:00 am,3:30 am,4:00 am,4:30 am,5:00 am,5:30 am,6:00 am,6:30 am,7:00 am,7:30 am,8:00 am,8:30 am,9:00 am,9:30 am,10:00 am,10:30 am,11:00 am,11:30 am,12:00 pm,12:30 pm,1:00 pm,1:30 pm,2:00 pm,2:30 pm,3:00 pm,3:30 pm,4:00 pm,4:30 pm,5:00 pm,5:30 pm,6:00 pm,6:30 pm,7:00 pm,7:30 pm,8:00 pm,8:30 pm,9:00 pm,9:30 pm,10:00 pm,10:30 pm,11:00 pm,11:30 pm,11:59 pm
0,dxjvx7,0,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,dxjvrw,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,dxjvlv,0,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3,dxjvi3,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,dxjv8u,2,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [9]:
# Add/rename column names for Sunday online users
df_Sunday_online_users.rename(columns = time_cols, inplace = True)

# Switch rows and columns
df_Sunday_online_users = df_Sunday_online_users.transpose()

# Drop first column since we are only interested in plotting the number of online users
df_Sunday_online_users = df_Sunday_online_users.drop('timestamp', axis=1)

df_Sunday_online_users.head()

,users_online
12:00 am,20128.0
12:30 am,21989.0
1:00 am,22101.0
1:30 am,25318.0
2:00 am,26671.0


We now do the same data cleaning for the remaining days of the week (Monday through Saturday).

In [10]:
# Clean Monday files
df_Monday_upvotes = pd.read_csv('Monday_votes.csv')
df_Monday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Monday_upvotes.rename(columns = time_cols, inplace = True)

df_Monday_comments = pd.read_csv('Monday_comments.csv')
df_Monday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Monday_comments.rename(columns = time_cols, inplace = True)

df_Monday_online_users = pd.read_csv('Monday_online_users.csv', index_col = 0)
df_Monday_online_users.rename(columns = time_cols, inplace = True)
df_Monday_online_users = df_Monday_online_users.transpose()
df_Monday_online_users = df_Monday_online_users.drop('timestamp', axis=1)

In [11]:
# Clean Tuesday files
df_Tuesday_upvotes = pd.read_csv('Tuesday_votes.csv')
df_Tuesday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Tuesday_upvotes.rename(columns = time_cols, inplace = True)

df_Tuesday_comments = pd.read_csv('Tuesday_comments.csv')
df_Tuesday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Tuesday_comments.rename(columns = time_cols, inplace = True)
df_Tuesday_online_users = pd.read_csv('Tuesday_online_users.csv', index_col = 0)
df_Tuesday_online_users.rename(columns = time_cols, inplace = True)
df_Tuesday_online_users = df_Tuesday_online_users.transpose()
df_Tuesday_online_users = df_Tuesday_online_users.drop('timestamp', axis=1)

In [12]:
# Clean Wednesday files
df_Wednesday_upvotes = pd.read_csv('Wednesday_votes.csv')
df_Wednesday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Wednesday_upvotes.rename(columns = time_cols, inplace = True)
df_Wednesday_comments = pd.read_csv('Wednesday_comments.csv')
df_Wednesday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Wednesday_comments.rename(columns = time_cols, inplace = True)

df_Wednesday_online_users = pd.read_csv('Wednesday_online_users.csv', index_col = 0)
df_Wednesday_online_users.rename(columns = time_cols, inplace = True)
df_Wednesday_online_users = df_Wednesday_online_users.transpose()
df_Wednesday_online_users = df_Wednesday_online_users.drop('timestamp', axis=1)

In [13]:
# Clean Thursday files
df_Thursday_upvotes = pd.read_csv('Thursday_votes.csv')
df_Thursday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Thursday_upvotes.rename(columns = time_cols, inplace = True)

df_Thursday_comments = pd.read_csv('Thursday_comments.csv')
df_Thursday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Thursday_comments.rename(columns = time_cols, inplace = True)

df_Thursday_online_users = pd.read_csv('Thursday_online_users.csv', index_col = 0)
df_Thursday_online_users.rename(columns = time_cols, inplace = True)
df_Thursday_online_users = df_Thursday_online_users.transpose()
df_Thursday_online_users = df_Thursday_online_users.drop('timestamp', axis=1)

In [14]:
# Clean Friday files
df_Friday_upvotes = pd.read_csv('Friday_upvotes.csv')
df_Friday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Friday_upvotes.rename(columns = time_cols, inplace = True)

df_Friday_comments = pd.read_csv('Friday_comments.csv')
df_Friday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Friday_comments.rename(columns = time_cols, inplace = True)

df_Friday_online_users = pd.read_csv('Friday_online.csv', index_col = 0)
df_Friday_online_users.rename(columns = time_cols, inplace = True)
df_Friday_online_users = df_Friday_online_users.transpose()
df_Friday_online_users = df_Friday_online_users.drop('timestamp', axis=1)

In [15]:
# Clean Saturday files
df_Saturday_upvotes = pd.read_csv('Saturday_votes.csv')
df_Saturday_upvotes.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Saturday_upvotes.rename(columns = time_cols, inplace = True)

df_Saturday_comments = pd.read_csv('Saturday_comments.csv')
df_Saturday_comments.rename(columns = {'Unnamed: 0': 'id'}, inplace=True)
df_Saturday_comments.rename(columns = time_cols, inplace = True)

df_Saturday_online_users = pd.read_csv('Saturday_online_users.csv', index_col = 0)
df_Saturday_online_users.rename(columns = time_cols, inplace = True)
df_Saturday_online_users = df_Saturday_online_users.transpose()
df_Saturday_online_users = df_Saturday_online_users.drop('timestamp', axis=1)

## Check Missing Values

Extracting data from the Reddit API most certainly outputs clean data values, as long as it is within the bounds of its api documentation. However, we will still check for any possible missing entries in the datasets as a precaution.

In [17]:
# Create a function to check the total number of missing values a specified dataframe has
def missing_values(df):
    if df.isnull().sum().sum() == 0:
        print('No missing value(s) detected in dataset.')

    else:
        total_number_missing = str(df.isnull().sum().sum())
        print('Missing value(s) detected in dataset: ' + total_number_missing)

# Check any missing values in all collected datasets
print('FOR SUNDAY DATASETS:')
missing_values(df_Sunday_upvotes)
missing_values(df_Sunday_comments)
missing_values(df_Sunday_online_users)

print('\n')

print('FOR MONDAY DATASETS:')
missing_values(df_Monday_upvotes)
missing_values(df_Monday_comments)
missing_values(df_Monday_online_users)

print('\n')

print('FOR TUESDAY DATASETS:')
missing_values(df_Tuesday_upvotes)
missing_values(df_Tuesday_comments)
missing_values(df_Tuesday_online_users)

print('\n')

print('FOR WEDNESDAY DATASETS:')
missing_values(df_Wednesday_upvotes)
missing_values(df_Wednesday_comments)
missing_values(df_Wednesday_online_users)

print('\n')

print('FOR THURSDAY DATASETS:')
missing_values(df_Thursday_upvotes)
missing_values(df_Thursday_comments)
missing_values(df_Thursday_online_users)

print('\n')

print('FOR FRIDAY DATASETS:')
missing_values(df_Friday_upvotes)
missing_values(df_Friday_comments)
missing_values(df_Friday_online_users)

print('\n')

print('FOR SATURDAY DATASETS:')
missing_values(df_Saturday_upvotes)
missing_values(df_Saturday_comments)
missing_values(df_Saturday_online_users)

FOR SUNDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.


FOR MONDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.


FOR TUESDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.


FOR WEDNESDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.


FOR THURSDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.


FOR FRIDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.


FOR SATURDAY DATASETS:
No missing value(s) detected in dataset.
No missing value(s) detected in dataset.
No missing va

Now that we have checked that none of our datasets contain any missing entries, we can get to modifying them for plotting.

## Find the Average Number of Upvotes

Before starting our analyses on the data, we need to find the average number of upvotes in all datasets corresponding to 30 minute intervals of the 24-hour period. We will use these averages as plot points for our line graphs for each day of the week.

In [ ]:
# Find the mean for all 30 minute time intervals for every day of the week
df_Sunday_upvotes.loc['mean'] = df_Sunday_upvotes.mean()
df_Monday_upvotes.loc['mean'] = df_Monday_upvotes.mean()
df_Tuesday_upvotes.loc['mean'] = df_Tuesday_upvotes.mean()
df_Wednesday_upvotes.loc['mean'] = df_Wednesday_upvotes.mean()
df_Thursday_upvotes.loc['mean'] = df_Thursday_upvotes.mean()
df_Friday_upvotes.loc['mean'] = df_Friday_upvotes.mean()
df_Saturday_upvotes.loc['mean'] = df_Saturday_upvotes.mean()

# Check the last few rows for Sunday
df_Sunday_upvotes.tail()

We are interested in plotting points for the average (mean) number of upvotes for each 30 minute interval, so we will be removing all rows and columns that are not relevant to plotting the average number of votes for the graphs. These extraneous data include the individual rows of upvotes for each 100 new posts from the r/funny subreddit. Since we will be plotting the averages, the NaN entry in the first column of the corresponding mean row will mess up the graphs during plotting. So, we will also do a removal of the id column.

In [ ]:
# Drop all rows except mean
df_Sunday_mean_upvotes = df_Sunday_upvotes.drop(df_Sunday_upvotes.index[0:100])
df_Monday_mean_upvotes = df_Monday_upvotes.drop(df_Monday_upvotes.index[0:100])
df_Tuesday_mean_upvotes = df_Tuesday_upvotes.drop(df_Tuesday_upvotes.index[0:100])
df_Wednesday_mean_upvotes = df_Wednesday_upvotes.drop(df_Wednesday_upvotes.index[0:100])
df_Thursday_mean_upvotes = df_Thursday_upvotes.drop(df_Thursday_upvotes.index[0:100])
df_Friday_mean_upvotes = df_Friday_upvotes.drop(df_Friday_upvotes.index[0:100])
df_Saturday_mean_upvotes = df_Saturday_upvotes.drop(df_Saturday_upvotes.index[0:100])

In [ ]:
# Drop the first column containing NaN value
df_Sunday_mean_upvotes.dropna(axis=1, how='any', inplace = True)
df_Monday_mean_upvotes.dropna(axis=1, how='any', inplace = True)
df_Tuesday_mean_upvotes.dropna(axis=1, how='any', inplace = True)
df_Wednesday_mean_upvotes.dropna(axis=1, how='any', inplace = True)
df_Thursday_mean_upvotes.dropna(axis=1, how='any', inplace = True)
df_Friday_mean_upvotes.dropna(axis=1, how='any', inplace = True)
df_Saturday_mean_upvotes.dropna(axis=1, how='any', inplace = True)

In [ ]:
# Switch rows and columns
df_Sunday_mean_upvotes = df_Sunday_mean_upvotes.transpose()
df_Monday_mean_upvotes = df_Monday_mean_upvotes.transpose()
df_Tuesday_mean_upvotes = df_Tuesday_mean_upvotes.transpose()
df_Wednesday_mean_upvotes = df_Wednesday_mean_upvotes.transpose()
df_Thursday_mean_upvotes = df_Thursday_mean_upvotes.transpose()
df_Friday_mean_upvotes = df_Friday_mean_upvotes.transpose()
df_Saturday_mean_upvotes = df_Saturday_mean_upvotes.transpose()

# Check the modified dataset for Sunday
df_Sunday_mean_upvotes.head()

## Find the Average Number of Comments

Similarly to finding the average number of upvotes outlined above, we will also find the averages for the number of comments for each 30 minute interval over 24 hours.

In [ ]:
# Find the mean for all 30 minute time intervals for every day of the week
df_Sunday_comments.loc['mean'] = df_Sunday_comments.mean()
df_Monday_comments.loc['mean'] = df_Monday_comments.mean()
df_Tuesday_comments.loc['mean'] = df_Tuesday_comments.mean()
df_Wednesday_comments.loc['mean'] = df_Wednesday_comments.mean()
df_Thursday_comments.loc['mean'] = df_Thursday_comments.mean()
df_Friday_comments.loc['mean'] = df_Friday_comments.mean()
df_Saturday_comments.loc['mean'] = df_Saturday_comments.mean()

In [ ]:
# Drop all rows except mean
df_Sunday_mean_comments = df_Sunday_comments.drop(df_Sunday_comments.index[0:100])
df_Monday_mean_comments = df_Monday_comments.drop(df_Monday_comments.index[0:100])
df_Tuesday_mean_comments = df_Tuesday_comments.drop(df_Tuesday_comments.index[0:100])
df_Wednesday_mean_comments = df_Wednesday_comments.drop(df_Wednesday_comments.index[0:100])
df_Thursday_mean_comments = df_Thursday_comments.drop(df_Thursday_comments.index[0:100])
df_Friday_mean_comments = df_Friday_comments.drop(df_Friday_comments.index[0:100])
df_Saturday_mean_comments = df_Saturday_comments.drop(df_Saturday_comments.index[0:100])

In [ ]:
# Drop the first column containing NaN value
df_Sunday_mean_comments.dropna(axis=1, how='any', inplace = True)
df_Monday_mean_comments.dropna(axis=1, how='any', inplace = True)
df_Tuesday_mean_comments.dropna(axis=1, how='any', inplace = True)
df_Wednesday_mean_comments.dropna(axis=1, how='any', inplace = True)
df_Thursday_mean_comments.dropna(axis=1, how='any', inplace = True)
df_Friday_mean_comments.dropna(axis=1, how='any', inplace = True)
df_Saturday_mean_comments.dropna(axis=1, how='any', inplace = True)

In [ ]:
# Switch rows and columns
df_Sunday_mean_comments = df_Sunday_mean_comments.transpose()
df_Monday_mean_comments = df_Monday_mean_comments.transpose()
df_Tuesday_mean_comments = df_Tuesday_mean_comments.transpose()
df_Wednesday_mean_comments = df_Wednesday_mean_comments.transpose()
df_Thursday_mean_comments = df_Thursday_mean_comments.transpose()
df_Friday_mean_comments = df_Friday_mean_comments.transpose()
df_Saturday_mean_comments = df_Saturday_mean_comments.transpose()

# Check the modified dataset for Sunday
df_Sunday_mean_comments.head()

# Data Analysis & Results

In [ ]:
#plot all upvotes
plt.figure(figsize = (12, 6))
plt.plot(df_Sunday_mean_upvotes, label = 'Sunday')
plt.plot(df_Monday_mean_upvotes, label = 'Monday')
plt.plot(df_Tuesday_mean_upvotes, label = 'Tuesday')
plt.plot(df_Wednesday_mean_upvotes, label = 'Wednesday')
plt.plot(df_Thursday_mean_upvotes, label = 'Thursday')
plt.plot(df_Friday_mean_upvotes, label = 'Friday')
plt.plot(df_Saturday_mean_upvotes, label = 'Saturday')
plt.xticks(np.arange(0, 49, 10))
plt.title('Average Number of Upvotes of all Days of the Week for First 100 New Posts in r/funny Subreddit')
plt.xlabel('Time')
plt.ylabel('Average Upvotes')
plt.legend(loc='upper left')

In [ ]:
#plot all comments
plt.figure(figsize = (12, 6))
plt.plot(df_Sunday_mean_comments, label = 'Sunday')
plt.plot(df_Monday_mean_comments, label = 'Monday')
plt.plot(df_Tuesday_mean_comments, label = 'Tuesday')
plt.plot(df_Wednesday_mean_comments, label = 'Wednesday')
plt.plot(df_Thursday_mean_comments, label = 'Thursday')
plt.plot(df_Friday_mean_comments, label = 'Friday')
plt.plot(df_Saturday_mean_comments, label = 'Saturday')
plt.xticks(np.arange(0, 49, 10))
plt.title('Average Number of Comments of all Days of the Week for First 100 New Posts in r/funny Subreddit')
plt.xlabel('Time')
plt.ylabel('Average Comments')
plt.legend(loc='lower right')

In [ ]:
#plot all online users
plt.figure(figsize = (12, 6))
plt.plot(df_Sunday_online_users, label = 'Sunday')
plt.plot(df_Monday_online_users, label = 'Monday')
plt.plot(df_Tuesday_online_users, label = 'Tuesday')
plt.plot(df_Wednesday_online_users, label = 'Wednesday')
plt.plot(df_Thursday_online_users, label = 'Thursday')
plt.plot(df_Friday_online_users, label = 'Friday')
plt.plot(df_Saturday_online_users, label = 'Saturday')
plt.xticks(np.arange(0, 49, 10))
plt.title('Hourly Number of Online Users of all Days of the Week in r/funny Subreddit')
plt.xlabel('Time')
plt.ylabel('Number of Users Online')
plt.legend(loc='upper left')

# Ethics & Privacy

   As discussed in the introduction of our research, the implicaitons of this data is great. Although there are many limitations in the methods of our data collection process, the concept of this research question is very important. Using the data from this research in other subreddits could lead to more serious effects on post interaction. Users could utilize these findings to strategically generate the most karma per post, click-farms can capitilize on this time efficacy, companies can maximize the monetary value of their marketing posts, and unfortunately, hategroups or negative propganda posts can also rely on this data for maximum viewer exposure and sharing potential.

   Additionally, although we do not collect any personally identifying information, the data collected is user generated and therefore may lead to a loss of confidentiality in ways we cannot forsee. For example, one issue we encountered was when we collected the current online active subscribers of r/funny; these username identifications are mostly annonymous but can inherently contain personally identifying aspects. Furthermore, if these research methods are expanded to mutiple subreddits, users can be cross idenified as being active in various subreddits that are personally identifying that can very well lead to loss of annomitiy. 

# Conclusion & Discussion

We hypothesised that the optimal time to post on the r/funny subreddit was Saturday morning since we assumed that the majority of users tend to access the application after a long week of labor/school. However, after data cleaning, analysis, and averaging the results, we happen to be quite wrong.

Thursday had the most average upvotes and average comments at around 12pm, during lunch time.

Because we only looked at r/funny, we believe that our analysis is not fully generalizable to all of reddit but it should give good insights and knowledge because r/funny is one of the bigger subreddits with over 27 million members. We could also improve our analysis in the future by tracking different posts at multiple times instead of only midnight so that we get a wider range of analysis for posts made at various times.

#### Upvote Data Results
From our upvotes data, it appears that posts made during midnight of Saturday and Sunday peaked around 11am. The two line graphs closely followed each other but Saturday posts had a higher average upvote count compared to Sunday. Wednesday and Thursday posts reach their peak around 12pm, lunch time. Monday, Tuesday, and Friday posts reach their peak around 6am. Ranking the days in from highest average upvotes to lowest: Thursday, Friday, Wednesday, Monday, Saturday, Sunday, Tuesday. So depending on the day, you should post 1-2 hours before the time of the peak upvotes for the corresponding day since that is when most upvotes are being given by users. After time of peak upvotes, the average number of upvotes given is close to 0 which is why we see a flat line afterwards.

In [ ]:
data = {'Monday': ['6 am', 4], 'Tuesday': ['6 am',7], 'Wednesday':['12 pm',3], 'Thursday':['12 pm',1], 'Friday':['6 am', 2], 'Saturday': ['11 am',5], 'Sunday':['11 am',6]}
T = pd.DataFrame.from_dict(data, orient='index', columns = ['Time of Peak Upvotes', 'Rank for Most Upvotes'])
T

#### Comment Data Results
The average number of comments graph had a similar form to the average number of upvotes graph. The ranking of days with the most comments was the same as the ranking for upvotes. The rate of comments being given to posts after they were posted at midnight roughly followed the same rate as upvotes. When upvotes were increasing rapidly, so were comments.

#### Online Users Data Results
It should be noted that there was a bug during collection of online users for Monday. The script was not correctly updating the values of online users so it kept appending the same value to the csv which is why we can see Monday as a straight line on the Online Users graph.

There are fluctuations in the amount of online users per day, but for most days the peak amount of online users lie between 9am and 4pm. Sunday peaks earlier at 7am and dips at 12am. Thursday has the absolute highest number of users on at a time. Sunday is second, but retains that amount of users for a longer time than Thursday.

In [ ]:
data = {'Monday': ['12 am'], 'Tuesday': ['3:30 pm'], 'Wednesday':['12:30 pm'], 'Thursday':['9:30 am'], 'Friday':['4 pm'], 'Saturday': ['10 am'], 'Sunday':['6:30 am']}
T = pd.DataFrame.from_dict(data, orient='index', columns = ['Time of Peak Online Users'])
T